In [10]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import pandas as pd
import numpy as np
import pyreadstat
import re


In [6]:
# Load Data
fpath = 'data/ATP W42.sav'

df, meta = pyreadstat.read_sav(fpath)

In [7]:
# Used meta.column_names to find questionnaire items with subject matter grouping (e.g. starts with 'RQ')
# Added to lists to use later on

past_future = ['PAST_W42', 'FUTURE_W42']
society = ['SC1_W42']
policy = [i for i in df.columns if 'POLICY' in i]
confidence = [i for i in df.columns if 'CONF' in i]
rq_form1 = [i for i in df.columns if 'RQ' in i]
pw_form2 = [i for i in df.columns if 'PQ' in i]
scm = [i for i in df.columns if 'SCM' in i]
q = [i for i in df.columns if re.search("^Q[0-9]", i)]
pop = [i for i in df.columns if 'POP' in i]
knowledge = [i for i in df.columns if 'KNOW' in i]
demographics = [i for i in df.columns if 'F_' in i]
weight = ['WEIGHT_W42']

In [8]:
# Dictionary of column names (abbreviations) and corresponding column values (instruction on questionnaire)
# Example

meta_dict = dict(zip(meta.column_names, meta.column_labels))

##### Example: 
meta_dict['SC1_W42']
##### Output:
'SC1. Overall, would you say science has had a mostly positive effect on our society or a mostly negative effect on our society?'

In [11]:
# Changes the values in the dataframe according to the value formats in the metadata.
# Because the data is Categorical or Ordinal, any functions we perform won't require the original float values

df_copy = pyreadstat.pyreadstat.set_value_labels(df, meta)

In [43]:
# This function returns a cross tabulation of weighted frequencies for the given variables

def crosstab_frequency(array1, array2):
    return pd.crosstab(df_copy[array1],
                df_copy[array2],
                df_copy.WEIGHT_W42, aggfunc = sum, dropna=True,
                normalize='columns').loc[meta.variable_value_labels[array1].values(), 
                                         meta.variable_value_labels[array2].values()]

In [3]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Science and Society"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])


# Run app and display result inline in the notebook
app.run_server(mode='inline')